In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

RAG - Reasoning Engine with Vertex AI Search
@forusone (shins777@gmail.com)


### Reasoning Engine in Vertex AI **Search**

### Install packages for Python

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    langchain-google-community \
    google-cloud-discoveryengine \
    google-api-python-client \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Initial set up

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
    grounding,
)

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Define tool(Vertex AI Search-Layout parser)

In [4]:
from vertexai.preview.generative_models import grounding

DATA_STORE_ID = "daou-search-ds-20240821_1724026314068"
DATA_STORE_LOCATION = "global"

tool = Tool.from_retrieval(
    grounding.Retrieval(
        grounding.VertexAISearch(
            project=PROJECT_ID,
            datastore=DATA_STORE_ID,
            location=DATA_STORE_LOCATION,
        )
    )
)

### Define prompt template

In [5]:
from googleapiclient import discovery
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.memory import ChatMessageHistory
from langchain_core import prompts

# Define prompt template
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Helper methods for chat history

In [6]:
# Initialize session history
store = {}

def get_session_history(session_id: str):
  """
    Get session history
  """
  if session_id not in store:
      store[session_id] = ChatMessageHistory()
  return store[session_id]

#---------------------------------------------------------------
def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

### Create reasoning engine for Vertex AI Search RAG.

In [ ]:
from vertexai.preview import reasoning_engines

agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=MODEL,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[tool],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
from time import perf_counter

session_id = "sess_101"

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 안녕하세요.


2.15373778600042

AI Agent : 안녕하세요! 

롯데리아는 8월 8일부터 버거류 20종을 포함한 일부 제품의 가격을 인상했습니다. '리아 불고기' 세트는 7,100원에 판매되고 있습니다. 

롯데리아 외에도 여러 프랜차이즈 업체들이 가격 인상을 단행했습니다. 스타벅스코리아는 지난달 31일 음료 가격을 대용량 기준으로 올렸습니다. 

가격 인상은 소비자들에게 부담으로 작용할 수 있지만, 원자재 가격 상승과 인건비 상승 등의 이유로 불가피한 측면도 있습니다. 

프랜차이즈 업체들은 가격 인상과 함께 고객 만족도를 높이기 위한 다양한 노력을 기울이고 있습니다. 

궁금한 점이 있으면 언제든지 물어보세요! 😊


------------------------------------ 
사용자: 인건비 상승이 왜 일어나 ?


3.3479005069993946

AI Agent : 인건비 상승은 여러 요인이 복합적으로 작용하여 발생합니다. 

* **최저임금 인상:** 정부는 매년 최저임금을 인상하고 있습니다. 최저임금 인상은 근로자들의 소득 증가에 기여하지만, 기업 입장에서는 인건비 부담 증가로 이어질 수 있습니다.
* **물가 상승:** 물가 상승은 생활비 증가로 이어지고, 근로자들은 더 많은 임금을 요구하게 됩니다. 이는 기업의 인건비 부담을 가중시킵니다.
* **인력 부족:** 인구 감소와 고령화로 인해 생산 가능 인구가 줄어들면서 인력 부족 현상이 심화되고 있습니다. 기업들은 인력 확보를 위해 임금을 높여야 하는 상황에 처하게 됩니다.
* **경쟁 심화:** 기업들은 우수한 인력을 확보하기 위해 경쟁적으로 임금을 높이고 있습니다. 이는 인건비 상승을 부추기는 요인이 됩니다.
* **근로 환경 개선 요구:** 근로자들은 더 나은 근무 환경과 복지 혜택을 요구하고 있습니다. 이러한 요구는 기업의 인건비 부담을 증가시킵니다.

인건비 상승은 기업의 이윤 감소로 이어질 수 있지만, 근로자들의 소득 증가와 소비 지출 증가를 통해 경제 활성화에 기여할 수도 있습니다. 

한국에서 인건비 상승은 특히 심각한 문제로 인식되고 있습니다. 한국은 높은 생산성에도 불구하고 인건비 수준이 선진국에 비해 낮은 편입니다. 이는 한국 기업들의 경쟁력을 약화시키는 요인으로 작용할 수 있습니다. 

정부는 인건비 상승 문제를 해결하기 위해 최저임금 인상 속도를 조절하고, 기업의 인건비 부담을 줄여주는 정책을 추진하고 있습니다. 또한, 생산성 향상을 위한 투자를 지원하고, 고용 창출을 위한 정책을 추진하고 있습니다.


------------------------------------ 
사용자: 종료
[]
